## selenium 실습 1

In [31]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup as bs
import time

In [7]:
# 출처 : https://www.youtube.com/watch?app=desktop&v=pj4GCSsIxSk

In [8]:
driver = wb.Chrome()
driver.get("https://map.kakao.com/")

In [9]:
# 1. 검색창의 위치를 알아오기
# 2. 광주맛집이라는 키워드를 입력
# 3. 검색버튼 실행

# 우측 상단 파란 이미지 클릭해주는 코드
# → 이렇게 해야 접근이 가능
img = driver.find_element(By.CSS_SELECTOR,"div.inner_coach_layer")
img.click()

In [10]:
# 카카오맵 검색창 id 가져온다
# id=search.keyword.query를 순수하게 문자열로 가져오기 위해 \ <- 넣어준다
# 선택자 . 키워드는 클래스값을 명시하는 방법
# "." 문자 점으로 인식하고 싶을 때는 \을 넣어주면 해결된다

# 현재 search에는 검색창의 위치가 담겨있다
search = driver.find_element(By.CSS_SELECTOR, "#search\.keyword\.query")

In [11]:
# search에 값을 보내겠다 (send_keys) -> 검색창에 값 들어감
search.send_keys("광주맛집")

In [12]:
# 검색 버튼
searchBtn = driver.find_element(By.CSS_SELECTOR, "#search\.keyword\.submit")
searchBtn.click()

In [13]:
# 드라이버에서 바로 find하지 않고, BeautifulSoup 객체화를 진행한 다음 데이터를 수집한다
# - 내가 화면에서 보고 있는 데이터를 BeautifulSoup 객체화

# BeautifulSoup 만들때 request에서 받아왔던 requset.text 정보, lxml이라는 파싱 방법 통해 문자 데이터를 컴퓨터가 알 수 있는 html 언어로 BeautifulSoup 객체화 진행했었다
# 그런데 request 통해 가져온 데이터 없지 않음? -> 맞다
# 현재 우리가 웹드라이버 통해 브라우저가 정보를 담고 있는 것이기 때문에 request 객체에 있는 데이터는 존재할 수 없다

# 그러면 bs() 안에 브라우저가 갖고 있는 페이지 정보를 넣어줘야 한다
# 이 브라우저가 뭐냐면 driver (현재 내가 보고 있는 지도 화면의 정보를 알고 있는 것이 driver 객체)
# BeautifulSoup과 Selenium 합쳐서 데이터를 가져온다

soup = bs(driver.page_source, "lxml")

In [14]:
# soup에서 데이터 가져오는 명령 soup.select
soup.select

<bound method Tag.select of <html class="index" lang="ko"><head>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="width=1024" name="viewport"/>
<link href="//t1.daumcdn.net/kakaomapweb/map/202404091402526/app_A.css" rel="stylesheet"/>
<link href="//t1.daumcdn.net/kakaomapweb/map/202404091402526/app_B.css" rel="stylesheet"/>
<link href="//t1.daumcdn.net/kakaomapweb/map/202404091402526/app_C.css" rel="stylesheet"/>
<link href="//t1.daumcdn.net/kakaomapweb/map/202404091402526/app_D.css" rel="stylesheet"/>
<link href="//t1.daumcdn.net/kakaomapweb/map/202404091402526/canvas_roadview.css" rel="stylesheet"/>
<link href="https://t1.daumcdn.net/kakaomapweb/subway/linemap/canvas/prod/css/subway.css" rel="stylesheet"/>
<link href="/favicon.ico" rel="icon" type="image/x-icon"/>
<title>카카오맵</title>
<script type="text/javascript">
if (false === true && true && document.referrer.split('?')[0] === "https://accounts.kakao.com/login?continue=".split('?')[0]) {
	window.location.href 

In [15]:
# 가게 이름은 a 태그의 class="link_name"
# 광주의 음식점 나온다
title = soup.select("a.link_name")

In [16]:
# 내가 필요한 것은 순수한 글자 정보
# 넘겨받은 타입 list, list는 크기가 정해져 있는 데이터라서 반복문 for 활용

for i in title:
    # 글자정보만 print
    print(i.text)

광주맛집
광주맛집 소신식당
광주휴대폰맛집
광주한우맛집 소신정육점
광주맛집효심싸닭갈비
광주누수맛집 라이동합설비
광주집
거진항맛집코다리네 광주본점
광주무등산
광주국밥
광주수산
진짜광주식당
광주식당
전통맛집할매순대국 광주초월점
광주똑순이아구찜


In [17]:
# 장소 더보기 버튼 클릭
# 장소 더보기 검사 - a 태그의 id="info.search.place.more"
btnMore = driver.find_element(By.CSS_SELECTOR, "#info\.search\.place\.more")
btnMore.click()
# 누르면 더보기 버튼 누르면서 한 페이지 넘어간다

In [19]:
# 이제 페이지가 1, 2, 3 ... 인지 번호 알아와서 데이터 클릭한 다음 수집 진행하는 반복문 코드 짜야한다
# 내가 몇 페이지 클릭하고 있는지 번호 값을 알아와야 함
# 페이지 번호 2에 우클릭 -> 검사
# 각각의 a 태그를 통해 페이지를 바꾸고 있다
# 우리에게 필요한 것은 1 ~ 5페이지까지 클릭할 수 있는 5개의 a 태그이다

# id는 중복 안되어서 전부 다르고, class는 INACTIVE, ACTIVE 클릭할 때마다 값이 바뀐다
# -> id와 class 값으로는 데이터를 한번에 불어오기 어렵다
# -> 이럴 때는 부모 태그를 본다

# a 태그의 부모 태그를 보면, div 태그 존재하고 class값으로 pageWrap 존재
# 그리고 이것의 부모도 마찬가지로 id값 갖고 있다 (info.search.page)

# 페이지 번호 데이터 수집
# 첫 번째 부모님은 div 태그, id="info.search.page"
# 그 밑에 있는 div를 갖고온 다음 그 밑의 a 태그를 불러오겠다고 접근하면 5개의 a 태그 한번에 불러올 수 있다

page = driver.find_elements(By.CSS_SELECTOR, "#info\.search\.page > div > a")

In [28]:
# 첫 번째 페이지 클릭한 다음, 글자 정보 수집하고 끝나면 두 번째 페이지 클릭해서 글자 수집
# 이렇게 클릭 - 들어가서 글자수집 반복

try:
    while True:
        for i in range(6):

            # 페이지 5번까지 갔으면 카카오맵의 ">" 클릭해줘야 한다
            # 조건 줘야한다. 만약 내가 클릭한 데이터가 5보다 크다면, 다음 버튼을 클릭해랴는 명령이 필요하다

            if(i < 5):

                # 5페이지 까지의 데이터 수집
            
                # page에 있는 i번째 a 태그 가져와서 click
                page[i].click()

                # 페이지가 바뀔 때는 time.sleep을 넣어서 페이지가 쉴 수 있고 데이터를 업로드하고 로딩할 수 있는 시간 벌어야 한다
                time.sleep(2)

                # 현재 페이지에 있는 정보를 새로 받아야 한다 -> soup 데이터로 바꿔줘야 한다 (데이터 바꿔주는 BeautifulSoup 코드)
                # 화면이 바뀌었기 때문에 최신화된 화면의 정보가 필요함 -> 매번 클릭할 때마다 새로운 soup 데이터 필요하다
                soup = bs(driver.page_source, "lxml")

                # 데이터 출력하는 문장이 필요하다
                title = soup.select("a.link_name")

                # 수집한 데이터를 출력
                for i in title:
                    print(i.text)
            
            else:
                #마지막 페이지 클릭했을 때 다음 버튼 클릭하는 로직
                # ">" : id값으로 info.search.page.next를 가진 button 태그
                # more = driver.find_element(By.CSS_SELECTOR, "info\.search\.page\.next")
                more = driver.find_element(By.XPATH,'//*[@id="info.search.page.next"]')
                more.click()
                time.sleep(2)
except:
    print("수집이 완료되었습니다!")

광주맛집
광주맛집 소신식당
광주휴대폰맛집
광주한우맛집 소신정육점
광주맛집효심싸닭갈비
광주누수맛집 라이동합설비
광주집
거진항맛집코다리네 광주본점
광주무등산
광주국밥
광주수산
진짜광주식당
광주식당
전통맛집할매순대국 광주초월점
광주똑순이아구찜
정가득광주본가
광주집
광주식당
광주찌개전문
광주곱창
광주집
카페 광주미용실
광주족발 서대문점
광주식당
광주식당
광주집순대국
광주집 보신탕
광주육전
광주곱창
광주식당
광주식당
광주집
광주회관
광주식당
광주식당
광주집
광주회집
이모네광주곱창
광주식당
광주 섬진강 논현본점
광주회관
청량리광주냉면
한우리광주집
광주식당
광주식당감자탕
광주먹자집
광주식당
광주홍어회찜
광주생오리돌구이전문
광주식당
광주식당
광주식당
광주가든
광주등심 (휴업중)
광주식당
광주한양식당
광주수산
광주집
군산광주회집
신은정광주소머리국밥
광주할머니곱창
가마솥광주통닭
광주사철탕삼계탕
광주집
광주식당
광주식당
광주식당
광주식당
광주함평호남횟집
광주진국밥
광주식당
안양중앙시장곱창골목광주집
광주식당
광주오리탕
광주오리탕
광주식당
광주장성오리탕
광주대명식당
광주식당
광주아구찜.탕
광주생오리돌구이 감일점
광주가든
광주나들목포차
광주식당
광주함평호남회집
광주식당
광주식당
광주신안월미회집
안양명물최고의맛광주집
전라도광주밥상 광주무등횟집
자성화맛집코다리네 광주점
광주머리고기집
광주집
신호남광주회집
내고향광주식당
광주집
광주식당
광주황제닭갈비
광주홍탁
광주소머리국밥
광주포차
광주분식
광주실내포차
광주뚝배기
광주칼국수 경기광주본점
남도맛집
광주공원
태전동맛집코다리네
음식의명가 먹거리마당맛집
광주막 본점
광주정육점식당
광주집
광주아구찜 북어해장국
빛고을광주식당
광주원조감자탕
광주식당
콩카페 경기광주 태전점
광주명가족발
오포자성화맛집코다리네
광주가마솥닭강정
남도맛집
광주공원
태전동맛집코다리네
음식의명가 먹거리마당맛집
광주막 본점
광주정육점식당
광주집
광주아구찜 북어해장국
빛고을광주식당
광주원조감자탕
광주식당
콩카페 경기광주 태전점
광주명가족발
오포자성화맛집코다리네
광주가마솥닭강정
광주초계

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=124.0.6367.60)
Stacktrace:
	GetHandleVerifier [0x00007FF69086F922+3045234]
	(No symbol) [0x00007FF690513652]
	(No symbol) [0x00007FF6903D7393]
	(No symbol) [0x00007FF690427292]
	(No symbol) [0x00007FF690419FDB]
	(No symbol) [0x00007FF690449DAA]
	(No symbol) [0x00007FF690419916]
	(No symbol) [0x00007FF690449FC0]
	(No symbol) [0x00007FF6904691C4]
	(No symbol) [0x00007FF690449B53]
	(No symbol) [0x00007FF69041813C]
	(No symbol) [0x00007FF690418D71]
	GetHandleVerifier [0x00007FF69089E89D+3237613]
	GetHandleVerifier [0x00007FF6908E0BA7+3508727]
	GetHandleVerifier [0x00007FF6908D976F+3478975]
	GetHandleVerifier [0x00007FF69064F1C6+815126]
	(No symbol) [0x00007FF69051ED0F]
	(No symbol) [0x00007FF690519854]
	(No symbol) [0x00007FF6905199E2]
	(No symbol) [0x00007FF69050A574]
	BaseThreadInitThunk [0x00007FF99EF07344+20]
	RtlUserThreadStart [0x00007FF9A06426B1+33]


In [ ]:
# 페이지 5번까지 갔으면 카카오맵의 ">" 클릭해줘야 한다
# 조건 줘야한다. 만약 내가 클릭한 데이터가 5보다 크다면, 다음 버튼을 클릭해랴는 명령이 필요하다

## selenium 실습 2

In [ ]:
# 출처 : https://coding-230227.tistory.com/70

# BeautifulSoup + Selenium을 같이 조화롭게 코드를 작성
# 왜?
    # Selenium은 브라우저 제어 등 자동화에 아주 특화
    # but Selenium 자체로 html 파싱, 데이터 추출에 어느 정도 한계가 있다. 
    # 그렇기 때문에 각자의 장점을 살려서, 짬뽕
    
    
    
# 파싱 (Parsing)이란?
# - 웹페이지에서 원하는 데이터를 추출, 가공하기 쉬운 상태로 변환하는 것
# - 예를 들면, 생선을 낚았다 최종적으로 통조림의 형태로 만들어야 하는데, 손질 하려는 절차를 파싱이라고 한다.

# 생선과는 달리, 웹페이지에 존재하는 데이터는 list, dictionary와 같은 자료구조와 다르다. 

# 그래서 parser라는 함수나 프로그램으로 다루기 쉬운 형태로 바꾸어 주는데

# 이 과정을 '파싱'이라고 한다.

In [60]:
# 라이브러리 import 

from selenium import webdriver as wb
from bs4 import BeautifulSoup as bs

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time 
from tqdm import tqdm
import pandas as pd

In [63]:
# 브라우저 할당 
driver = wb.Chrome()
driver.get('https://map.kakao.com/')
# driver.maximize_window()

In [64]:
# 파란색 팝업 창 기억나는가?
    # 이 부분을 사용자가 클릭해서 없애듯, 
    # 컴퓨터에게도 똑같이 시켜줘야 한다.
    
click_blue_popup = driver.find_element(By.CSS_SELECTOR,'body > div.coach_layer.coach_layer_type1 > div > div > div > span')
click_blue_popup.click()

In [65]:
# 검색창 지정 (마우스로 올려놓기)

search = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.query')

# 검색어 입력('이수역 맛집')

search.send_keys('이수역 맛집')

# ENTER키 누르고 데이터가 로드할 수 있는 시간까지 할당해주기

search.send_keys(Keys.ENTER)
time.sleep(3)

In [29]:
# BeautifulSoup으로 객체화

In [66]:
# 보다 더 빠른 데이터 수집을 위해 -> 추출하는 시간도 이제는 고려

soup = bs(driver.page_source,'lxml')
soup

<html class="index" lang="ko"><head>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="width=1024" name="viewport"/>
<link href="//t1.daumcdn.net/kakaomapweb/map/202404231316299/app_A.css" rel="stylesheet"/>
<link href="//t1.daumcdn.net/kakaomapweb/map/202404231316299/app_B.css" rel="stylesheet"/>
<link href="//t1.daumcdn.net/kakaomapweb/map/202404231316299/app_C.css" rel="stylesheet"/>
<link href="//t1.daumcdn.net/kakaomapweb/map/202404231316299/app_D.css" rel="stylesheet"/>
<link href="//t1.daumcdn.net/kakaomapweb/map/202404231316299/canvas_roadview.css" rel="stylesheet"/>
<link href="https://t1.daumcdn.net/kakaomapweb/subway/linemap/canvas/prod/css/subway.css" rel="stylesheet"/>
<link href="/favicon.ico" rel="icon" type="image/x-icon"/>
<title>카카오맵</title>
<script type="text/javascript">
if (false === true && true && document.referrer.split('?')[0] === "https://accounts.kakao.com/login?continue=".split('?')[0]) {
	window.location.href = window.location.origin + '

In [67]:
# 식당이름 (titles)

titles = soup.select('a.link_name')

for i in titles:
    print(i.text)

애플하우스 이수
그릴진
세녹
소담촌 사당직영점
전주전집
스시로로
방배김밥
호요
원조쯔왕돈까스
남미플랜트랩
단아한정식
신사펍 이수점
낙성곱창
삼육가 사당역1호점
유정우함흥냉면


In [68]:
# 식당 주소 -> addresses

addresses = soup.select('div.addr > p:nth-child(1)')
# addresses = soup.select('div.addr > p:nth-child(2)')


for i in addresses:
    print(i.text)

서울 동작구 동작대로27다길 29 서광시티뷰 2층 201호
서울 동작구 동작대로27다길 31 세양그레이스빌 1층 101호
서울 동작구 사당로 304
서울 동작구 동작대로 43 5층
서울 동작구 동작대로7길 19
서울 동작구 동작대로23길 29
서울 동작구 동작대로27길 59-16 1층
서울 동작구 동작대로13길 6-7 1층
서울 동작구 동작대로23길 33 1,2층
서울 서초구 방배천로4안길 55 2층
서울 동작구 동작대로27가길 44 2층
서울 동작구 동작대로 129 4층
서울 동작구 동작대로19길 60
서울 서초구 방배천로 10 동화빌딩 1층
서울 동작구 동작대로29다길 7-1 1층


In [32]:
# 더보기 버튼 지정 및 클릭 
    # 기능적인 부분이니까 XPATH -> 동적인 요소 selenium 사용

click_more_location = driver.find_element(By.XPATH,'//*[@id="info.search.place.more"]')
click_more_location.click()
time.sleep(2)

In [33]:
# 더보기 버튼을 누른 후에는 , 번호 버튼들이 존재
    # 해당 항목들을 수집
page = driver.find_elements(By.CSS_SELECTOR,'#info\.search\.page > div > a')
page

[<selenium.webdriver.remote.webelement.WebElement (session="647977b4e338ce9ba6af344d09f5aa72", element="f.A9F36142300E47D78AD174D3A7155BE4.d.5BA661E3A80989E0C6DB95AD21E3035F.e.316")>,
 <selenium.webdriver.remote.webelement.WebElement (session="647977b4e338ce9ba6af344d09f5aa72", element="f.A9F36142300E47D78AD174D3A7155BE4.d.5BA661E3A80989E0C6DB95AD21E3035F.e.317")>,
 <selenium.webdriver.remote.webelement.WebElement (session="647977b4e338ce9ba6af344d09f5aa72", element="f.A9F36142300E47D78AD174D3A7155BE4.d.5BA661E3A80989E0C6DB95AD21E3035F.e.318")>,
 <selenium.webdriver.remote.webelement.WebElement (session="647977b4e338ce9ba6af344d09f5aa72", element="f.A9F36142300E47D78AD174D3A7155BE4.d.5BA661E3A80989E0C6DB95AD21E3035F.e.319")>,
 <selenium.webdriver.remote.webelement.WebElement (session="647977b4e338ce9ba6af344d09f5aa72", element="f.A9F36142300E47D78AD174D3A7155BE4.d.5BA661E3A80989E0C6DB95AD21E3035F.e.320")>]

In [ ]:
# 크롤러 조각하기

In [34]:
# 빈 리스트 2개를 생성 


titles_list = []
addresses_list = []

try : # 시도해봐! 뭘? 아래 실행코드들을
    while True: # 조건이 참일 경우를 
        for i in tqdm(range(6)):
            # 만약 순차가 5 미만일 경우 
            if i < 5:
                # page 클릭
                # page = driver.find_elements(By.CSS_SELECTOR,'#info\.search\.page > div > a')
                    # find_elements 는 요소를 지정해주는 기능이지 값을 저장하는 함수가 아니다! 그래서 for문 반복문에 넣을 필요없음
                page[i].click()
                time.sleep(2)
                
                # beautifulsoup 객체화
                soup = bs(driver.page_source,'lxml')
                
                # 식당이름 
                    # 변수 = soup.select 선택한 데이터 값을 저장하는 함수! 그래서 페이지가 바뀔 때마다 새로 선언해줘야함
                titles = soup.select('a.link_name')
                
                # for문을 돌면서 titles에서 순차적으로 
                for i in titles :
                    # titles_list에 순차적으로 뽑은 요소의 텍스트만 넣어줘
                    titles_list.append(i.text)
                
                # 식당주소
                    # 변수 = soup.select 선택한 데이터 값을 저장하는 함수! 그래서 페이지가 바뀔 때마다 새로 선언해줘야함
                addresses = soup.select('div.addr > p:nth-child(1)')
                
                # for문을 돌면서 addresses에서 순차적으로 
                for i in addresses :
                    # addresses_list에 순차적으로 뽑은 요소의 텍스트만 넣어줘
                    addresses_list.append(i.text)
            else:
                # 그렇지 않은 경우는,
                # i가 5보다 큰 경우
                # 5번 버튼까지 다 돌고, 다음 버튼을 눌러야하는 경우
                click_next = driver.find_element(By.XPATH,'//*[@id="info.search.page.next"]')
                click_next.click()
                time.sleep(3)
except:
    print("수집이 완료되었습니다.")

driver.quit()
len(titles_list), len(addresses_list)

 67%|██████▋   | 4/6 [00:08<00:04,  2.22s/it]


수집이 완료되었습니다.


(500, 500)

In [35]:
addresses_list

['서울 동작구 동작대로27다길 29 서광시티뷰 2층 201호',
 '서울 동작구 동작대로27다길 31 세양그레이스빌 1층 101호',
 '서울 동작구 사당로 304',
 '서울 동작구 동작대로 43 5층',
 '서울 동작구 동작대로7길 19',
 '서울 동작구 동작대로23길 29',
 '서울 동작구 동작대로27길 59-16 1층',
 '서울 동작구 동작대로13길 6-7 1층',
 '서울 동작구 동작대로23길 33 1,2층',
 '서울 서초구 방배천로4안길 55 2층',
 '서울 동작구 동작대로27가길 44 2층',
 '서울 동작구 동작대로 129 4층',
 '서울 동작구 동작대로19길 60',
 '서울 서초구 방배천로 10 동화빌딩 1층',
 '서울 동작구 동작대로29다길 7-1 1층',
 '서울 서초구 동작대로 86 1층',
 '서울 동작구 동작대로27나길 8',
 '서울 서초구 방배천로18길 10 1층',
 '서울 동작구 동작대로27길 9 2층',
 '서울 동작구 남부순환로271길 24 1층',
 '서울 동작구 동작대로 71 1층',
 '서울 동작구 동작대로27가길 41 1층',
 '서울 서초구 방배천로 92 세앙아르비체 102동 107,108호',
 '서울 동작구 동작대로9길 4 1층',
 '서울 서초구 방배천로4안길 20 2층',
 '서울 서초구 방배천로 12 1층',
 '서울 서초구 방배천로 92 세양아르비채 102동 1층 109~112호',
 '서울 동작구 동작대로27길 31 2층',
 '서울 서초구 동작대로 86 동주빌딩 지하 1층',
 '서울 서초구 방배천로 22 1층',
 '서울 동작구 동작대로27다길 9',
 '서울 동작구 동작대로27길 38 1층 103,104호',
 '서울 동작구 동작대로27길 22 2층',
 '서울 서초구 서초대로 3-4 방배디오슈페리움1 1층 106호',
 '서울 동작구 동작대로29길 8 2층',
 '서울 동작구 동작대로5길 8 2층',
 '서울 동작구 사당로30길 19',
 '서울 서초구 방배천로2길 7 1층',
 

In [ ]:
# 빈 리스트 2개를 생성 


titles_list = []
addresses_list = []

try : # 시도해봐! 뭘? 아래 실행코드들을
    while True: # 조건이 참일 경우를 
        for i in tqdm(range(6)):
            # 만약 순차가 5 미만일 경우 
            if i < 5:
                # page 클릭
                # page = driver.find_elements(By.CSS_SELECTOR,'#info\.search\.page > div > a')
                    # find_elements 는 요소를 지정해주는 기능이지 값을 저장하는 함수가 아니다! 그래서 for문 반복문에 넣을 필요없음
                page[i].click()
                time.sleep(2)
                
                # beautifulsoup 객체화
                soup = bs(driver.page_source,'lxml')
                
                # 식당이름 
                    # 변수 = soup.select 선택한 데이터 값을 저장하는 함수! 그래서 페이지가 바뀔 때마다 새로 선언해줘야함
                titles = soup.select('a.link_name')
                
                # for문을 돌면서 titles에서 순차적으로 
                for i in titles :
                    # titles_list에 순차적으로 뽑은 요소의 텍스트만 넣어줘
                    titles_list.append(i.text)
                
                # 식당주소
                    # 변수 = soup.select 선택한 데이터 값을 저장하는 함수! 그래서 페이지가 바뀔 때마다 새로 선언해줘야함
                addresses = soup.select('div.addr > p:nth-child(1)')
                
                # for문을 돌면서 addresses에서 순차적으로 
                for i in addresses :
                    # addresses_list에 순차적으로 뽑은 요소의 텍스트만 넣어줘
                    addresses_list.append(i.text)
            else:
                # 그렇지 않은 경우는,
                # i가 5보다 큰 경우
                # 5번 버튼까지 다 돌고, 다음 버튼을 눌러야하는 경우
                click_next = driver.find_element(By.XPATH,'//*[@id="info.search.page.next"]')
                click_next.click()
                time.sleep(3)
except:
    print("수집이 완료되었습니다.")

driver.quit()
len(titles_list), len(addresses_list)

## 크롤링 - 강남역 고깃집

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from selenium import webdriver #pip install selenium
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re
import time
import selenium
selenium.__version__ #4.3.0

'4.19.0'

In [37]:
# https://daily-develope.tistory.com/563

source_url = "https://map.kakao.com/"
driver = wb.Chrome()
driver.get(source_url)

# id="search.keyword.query" 태그 선택=> input 태그 
searchbox=driver.find_element(By.ID,"search.keyword.query") 
searchbox.send_keys("강남역 고기집") #강남역 고기집 값입력

# id="search.keyword.submit" 태그 선택=> input 태그 
searchbutton=driver.find_element(By.ID,"search.keyword.submit") 
#script를 실행
#arguments[0].click(); : 첫번째 매개변수값을 클릭
driver.execute_script("arguments[0].click();", searchbutton) #검색버튼클릭.
time.sleep(2) #2초동안 대기 => 강남역 고기집 검색 결과가 브라우저에 표시됨

In [38]:
html = driver.page_source # 현재브라우저의 소스(html)
soup = bs(html, "html.parser") # BeautifulSoup 분석
# name="a": a태그
# attrs={"class":"moreview" : a태그중에 class속성이 moreview인 태그
moreviews = soup.find_all(name="a", attrs={"class":"moreview"}) # 상세보기들
page_urls = [] 
for moreview in moreviews:
    # moreview: 한개의 상세보기. a태그
    page_url = moreview.get("href") # a태그의 href 속성의 값
    page_urls.append(page_url) #상세보기의 href 속성값들 목록. 강남역 고기집의 URL 정보 목록
driver.close()    
print(page_urls)
print(len(page_urls))

['https://place.map.kakao.com/95713992', 'https://place.map.kakao.com/1503746075', 'https://place.map.kakao.com/1052874675', 'https://place.map.kakao.com/2011092566', 'https://place.map.kakao.com/168079537', 'https://place.map.kakao.com/26431943', 'https://place.map.kakao.com/741391811', 'https://place.map.kakao.com/26573718', 'https://place.map.kakao.com/1511737742', 'https://place.map.kakao.com/785573131', 'https://place.map.kakao.com/27511865', 'https://place.map.kakao.com/279490770', 'https://place.map.kakao.com/855757075', 'https://place.map.kakao.com/27238067', 'https://place.map.kakao.com/1780387311']
15


In [58]:
# 상세보기에 조회된 고기집 목록을 조회
columns = ['score', 'review']
df = pd.DataFrame(columns=columns) # 컬럼만 조회
driver = wb.Chrome() # 브라우저 실행

for page in page_urls:
    driver.get(page) # 고기집 상세화면
    time.sleep(2) # 2초 쉬기
    html = driver.page_source # html 소스 데이터 (page 소스를 html에 세팅)
    soup = bs(html, 'html.parser')
    try:
        total_strong = soup.find(name="strong", attrs={"class": "total_evaluation"})
        tot_num = total_strong.find(name="span", attrs={"class": "color_b"}) # 등록된 리뷰 갯수
        contents_div = soup.find(name='div', attrs={"class": "evaluation_review"}) #리뷰 영역. 별점, 내용
        rates=contents_div.find_all(name="em", attrs={"class": "num_rate"}) # 별점값 목록
        reviews = contents_div.find_all(name="p", attrs={"class": "txt_comment"}) # 리뷰 목록
    except:
        continue # try 구문에서 오류 발생된 경우 다음 페이지로 이동
        
    # 리뷰의 1페이지 정보를 df에 저장
    for rate, review in zip(rates, reviews): # (별점목록, 리뷰목록)
        row = [rate.text[0], review.find(name="span").text]
        series = pd.Series(row, index=df.columns) # score, review 인덱스 설정. 시리즈 객체로 생성.
        df = df.append(series, ignore_index=True)
        
    #tot_num.text : 전체 등록된 리뷰의 건수. (ex.197건)
    page_num = int(tot_num.text) // 5 + 1 # 197 // 5 = 39.4
    for button_num in range(2, page_num + 1): # 2페이지 ~ page_num 까지
        try:
            # 페이지 숫자의 테그
            another_reviews = driver.find_element("xpath", "//a[@data-page='" + str(button_num)+"']")
            another_reviews.click() # 페이지 숫자 클릭
            time.sleep(2)
            html = driver.page_source
            soup = bs(html, 'html.parser')
            contents_div = soup.find(name = "div", attrs={"class": "evaluation_review"})
            rates = contents_div.find_all(name="em", attrs={"class": "num_rate"}) # 별점목록
            reviews = contents_div.find_all(name="p", attrs={"class": "txt_comment"}) # 리뷰목록
            
            for rate, review in zip(rates, reviews):
                row=[rate.text[0], review.find(name="span").text]
                series = pd.Series(row, index=df.columns)
                df = df.append(series, ignore_index=True)
        except:
           break
driver.close()

In [40]:
df.info() #719
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   score   0 non-null      object
 1   review  0 non-null      object
dtypes: object(2)
memory usage: 124.0+ bytes


,score,review


In [52]:
rates

[]

In [85]:
contents_div.find_all(name="p", attrs={"class": "txt_comment"})

[<p class="txt_comment"><span></span><button class="btn_fold" type="button">더보기</button></p>,
 <p class="txt_comment txt_fold"><span>맛<br/>: 맛맛꿀마<br/><br/>스타일<br/>: 맛맛꿀마<br/><br/>식당<br/>: 맛꿀마<br/><br/>가격<br/>: 맛꿀마</span><button class="btn_fold" type="button">더보기</button></p>,
 <p class="txt_comment txt_fold"><span>강남최고의 갓성비집. <br/><br/>돈까스랑 (고기먹고볶아먹는)볶음밥맛집. 하이볼은 노맛. <br/><br/>자리꽤많고 방쪽은 예약만 받는듯. 일찍닫아서 항상 마지막까지있는 손님됨...ㅎㅎ...! <br/><br/>술.물.음료.반찬.수저는 알아서 가져다먹으면되고, 내부에 화장실있음.</span><button class="btn_fold" type="button">더보기</button></p>]

In [28]:
########## 1. 카카오맵을 크롤링하여 맛집리뷰에 사용되는 용어 분석하기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from selenium import webdriver #pip install selenium
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re
import time
import selenium
selenium.__version__ #4.3.0

#  = 'C:/KIC/chromedriver_win32/chromedriver.exe'
source_url = "https://map.kakao.com/"
driver = webdriver.Chrome()
driver.get(source_url)
# id="search.keyword.query" 태그 선택=> input 태그 
searchbox=driver.find_element(By.ID,"search.keyword.query") 
searchbox.send_keys("강남역 고기집") #강남역 고기집 값입력
# id="search.keyword.submit" 태그 선택=> input 태그 
searchbutton=driver.find_element(By.ID,"search.keyword.submit") 
#script를 실행
#arguments[0].click(); : 첫번째 매개변수값을 클릭
driver.execute_script("arguments[0].click();", searchbutton) #검색버튼클릭.
time.sleep(2) #2초동안 대기 => 강남역 고기집 검색 결과가 브라우저에 표시됨

html = driver.page_source # 현재브라우저의 소스(html)
soup = BeautifulSoup(html, "html.parser") # BeautifulSoup 분석
# name="a": a태그
# attrs={"class":"moreview" : a태그중에 class속성이 moreview인 태그
moreviews = soup.find_all(name="a", attrs={"class":"moreview"}) # 상세보기들
page_urls = [] 
for moreview in moreviews:
    # moreview: 한개의 상세보기. a태그
    page_url = moreview.get("href") # a태그의 href 속성의 값
    page_urls.append(page_url) #상세보기의 href 속성값들 목록. 강남역 고기집의 URL 정보 목록
driver.close()    
print(page_urls)
print(len(page_urls))

# 상세보기에 조회된 고기집 목록을 조회
columns = ['score', 'review']
df = pd.DataFrame(columns=columns) # 컬럼만 조회
driver = webdriver.Chrome() # 브라우저 실행

for page in page_urls:
    driver.get(page) # 고기집 상세화면
    time.sleep(2) # 2초 쉬기
    html = driver.page_source # html 소스 데이터 (page 소스를 html에 세팅)
    soup = BeautifulSoup(html, 'html.parser')
    try:
        total_strong = soup.find(name="strong", attrs={"class": "total_evaluation"})
        tot_num = total_strong.find(name="span", attrs={"class": "color_b"}) # 등록된 리뷰 갯수
        contents_div = soup.find(name='div', attrs={"class": "evaluation_review"}) #리뷰 영역. 별점, 내용
        rates=contents_div.find_all(name="em", attrs={"class": "num_rate"}) # 별점값 목록
        reviews = contents_div.find_all(name="p", attrs={"class": "txt_comment"}) # 리뷰 목록
    except:
        continue # try 구문에서 오류 발생된 경우 다음 페이지로 이동
        
    # 리뷰의 1페이지 정보를 df에 저장
    for rate, review in zip(rates, reviews): # (별점목록, 리뷰목록)
        row = [rate.text[0], review.find(name="span").text]
        series = pd.Series(row, index=df.columns) # score, review 인덱스 설정. 시리즈 객체로 생성.
        df = df.append(series, ignore_index=True)
        
    # #tot_num.text : 전체 등록된 리뷰의 건수. (ex.197건)
    # page_num = int(tot_num.text) // 5 + 1 # 197 // 5 = 39.4
    # for button_num in range(2, page_num + 1): # 2페이지 ~ page_num 까지
    #     try:
    #         # 페이지 숫자의 테그
    #         time.sleep(2)
    #         another_reviews = driver.find_element("x", "//a[@data-page='" + str(button_num)+"']")
    #         another_reviews.click() # 페이지 숫자 클릭
    #         time.sleep(2)
    #         html = driver.page_source
    #         soup = BeautifulSoup(html, 'html.parser')
    #         contents_div = soup.find(name = "div", attrs={"class": "evaluation_review"})
    #         rates = contents_div.find_all(name="em", attrs={"class": "num_rate"}) # 별점목록
    #         reviews = contents_div.find_all(name="p", attrs={"class": "txt_comment"}) # 리뷰목록
            
    #         for rate, review in zip(rates, reviews):
    #             row=[rate.text[0], review.find(name="span").text]
    #             series = pd.Series(row, index=df.columns)
    #             df = df.append(series, ignore_index=True)
    #     except:
    #        break
driver.close()

df.info() #719
df.head()


['https://place.map.kakao.com/95713992', 'https://place.map.kakao.com/1503746075', 'https://place.map.kakao.com/1052874675', 'https://place.map.kakao.com/2011092566', 'https://place.map.kakao.com/168079537', 'https://place.map.kakao.com/26431943', 'https://place.map.kakao.com/741391811', 'https://place.map.kakao.com/785573131', 'https://place.map.kakao.com/26573718', 'https://place.map.kakao.com/1511737742', 'https://place.map.kakao.com/279490770', 'https://place.map.kakao.com/27511865', 'https://place.map.kakao.com/27238067', 'https://place.map.kakao.com/855757075', 'https://place.map.kakao.com/1780387311']
15
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   score   0 non-null      object
 1   review  0 non-null      object
dtypes: object(2)
memory usage: 124.0+ bytes


,score,review


In [ ]:
# https://cordingdoah.tistory.com/97



## 오레노 라멘

In [1]:
# https://landshire.tistory.com/67

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

from selenium import webdriver
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import re
import time
from selenium.webdriver.common.keys import Keys
import math

# path = 'chromedriver'
# excutable_path = 'chromedriver.exe'
source_url = "https://map.kakao.com/"
driver = webdriver.Chrome()

driver.get(source_url)

search=driver.find_element(By.XPATH, '//*[@id="search.keyword.query"]')
search.send_keys("합정 라멘")
search.send_keys(Keys.ENTER)

time.sleep(2)

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
moreviews = soup.find_all(name="a", attrs={"class":"moreview"})

driver.close()

urls=[]
for i in moreviews:
    page_url=i.get("href")
    urls.append(page_url)
    
ramen=urls[0]

In [2]:
ramen

'https://place.map.kakao.com/1916682638'

In [3]:
columns = ['score', 'review']
df = pd.DataFrame(columns=columns)

stars=[]
reviews=[]

driver = webdriver.Chrome()
driver.get(ramen)

last_height = driver.execute_script("return document.body.scrollHeight")

while True:
            
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    time.sleep(2)

            # 스크롤 다운 후 스크롤 높이 다시 가져옴
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

    tmp=driver.page_source
    tmp2 = BeautifulSoup(tmp, "html.parser")

    # total_reviews=int(tmp2.select('#mArticle > div.cont_evaluation > div.evaluation_sorting > a > span.color_b')[0].get_text())
    total_reviews=int(tmp2.select('#mArticle > div.cont_evaluation > strong.total_evaluation > span.color_b')[0].get_text())

    pages=math.ceil(total_reviews/5)
    print(pages)
    time.sleep(5)

for i in range(1,pages+1):
    t1=driver.page_source
    t2=BeautifulSoup(t1, "html.parser")
    t3=t2.find(name="div", attrs={"class":"evaluation_review"})

    star=t3.find_all('em',{'class':'num_rate'})
    review=t3.find_all('p',{'class':'txt_comment'})

    stars.extend(star)
    reviews.extend(review)

    i=i+1
                
    if i > pages:
        break
    
    time.sleep(3)
    next_page = driver.find_element(By.XPATH, "//a[@data-page='" + str(i) + "']")
    # next_page = driver.find_element(By.XPATH, "//*[@data-page='" + str(i) + "']")

    next_page.send_keys(Keys.ENTER)
    time.sleep(5)
    
driver.close()

columns = ['score', 'review']
df = pd.DataFrame(columns=columns)
for s, r in zip(stars, reviews):
    row = [s.text[0], r.find(name="span").text]
    series = pd.Series(row, index=df.columns)
    df = df.append(series, ignore_index=True)
    
# df.to_csv('oreno_ramen.csv',encoding='utf-8-sig')

141


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[@data-page='2']"}
  (Session info: chrome=124.0.6367.61); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF673FBF922+3045234]
	(No symbol) [0x00007FF673C63652]
	(No symbol) [0x00007FF673B27564]
	(No symbol) [0x00007FF673B75F9D]
	(No symbol) [0x00007FF673B7606C]
	(No symbol) [0x00007FF673BBBDD7]
	(No symbol) [0x00007FF673B99DEF]
	(No symbol) [0x00007FF673BB91C4]
	(No symbol) [0x00007FF673B99B53]
	(No symbol) [0x00007FF673B6813C]
	(No symbol) [0x00007FF673B68D71]
	GetHandleVerifier [0x00007FF673FEE89D+3237613]
	GetHandleVerifier [0x00007FF674030BA7+3508727]
	GetHandleVerifier [0x00007FF67402976F+3478975]
	GetHandleVerifier [0x00007FF673D9F1C6+815126]
	(No symbol) [0x00007FF673C6ED0F]
	(No symbol) [0x00007FF673C69854]
	(No symbol) [0x00007FF673C6

## 내가 만들어봄

In [244]:
# 라이브러리 import 

from selenium import webdriver as wb
from bs4 import BeautifulSoup as bs

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time 
from tqdm import tqdm
import pandas as pd

In [248]:
# 브라우저 할당 
driver = wb.Chrome()
driver.get('https://map.kakao.com/')
# driver.maximize_window()

In [249]:
# 파란색 팝업 창 기억나는가?
    # 이 부분을 사용자가 클릭해서 없애듯, 
    # 컴퓨터에게도 똑같이 시켜줘야 한다.
    
click_blue_popup = driver.find_element(By.CSS_SELECTOR,'body > div.coach_layer.coach_layer_type1 > div > div > div > span')
click_blue_popup.click()

In [250]:
# 검색창 지정 (마우스로 올려놓기)

search = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.query')

# 검색어 입력('이수역 맛집')

store_name = '이도곰탕 본점'

search.send_keys(store_name)

# ENTER키 누르고 데이터가 로드할 수 있는 시간까지 할당해주기

search.send_keys(Keys.ENTER)
time.sleep(2)

In [251]:
# BeautifulSoup으로 객체화
# 보다 더 빠른 데이터 수집을 위해 -> 추출하는 시간도 이제는 고려

soup = bs(driver.page_source,'lxml')
soup

<html class="index" lang="ko"><head>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="width=1024" name="viewport"/>
<link href="//t1.daumcdn.net/kakaomapweb/map/202404231316299/app_A.css" rel="stylesheet"/>
<link href="//t1.daumcdn.net/kakaomapweb/map/202404231316299/app_B.css" rel="stylesheet"/>
<link href="//t1.daumcdn.net/kakaomapweb/map/202404231316299/app_C.css" rel="stylesheet"/>
<link href="//t1.daumcdn.net/kakaomapweb/map/202404231316299/app_D.css" rel="stylesheet"/>
<link href="//t1.daumcdn.net/kakaomapweb/map/202404231316299/canvas_roadview.css" rel="stylesheet"/>
<link href="https://t1.daumcdn.net/kakaomapweb/subway/linemap/canvas/prod/css/subway.css" rel="stylesheet"/>
<link href="/favicon.ico" rel="icon" type="image/x-icon"/>
<title>카카오맵</title>
<script type="text/javascript">
if (false === true && true && document.referrer.split('?')[0] === "https://accounts.kakao.com/login?continue=".split('?')[0]) {
	window.location.href = window.location.origin + '

In [252]:
# 상세보기
moreviews = soup.find_all(name="a", attrs={"class":"moreview"})

# 가장 먼저 나오는 검색 결과의 page url 저장
page_url = moreviews[0].get("href")
driver.close()
print(page_url)


https://place.map.kakao.com/14569639


In [267]:
# 상세보기에 조회된 고기집 목록을 조회
# columns = ['score', 'review']
# df = pd.DataFrame(columns=columns) # 컬럼만 조회
driver = wb.Chrome() # 브라우저 실행
driver.get(page_url) # 음식점 상세 화면
time.sleep(2) # 2초 쉬기

soup = bs(driver.page_source,'lxml')
total_strong = soup.find(name="strong", attrs={"class": "total_evaluation"})
tot_num = total_strong.find(name="span", attrs={"class": "color_b"}) # 등록된 리뷰 갯수
contents_div = soup.find(name='div', attrs={"class": "evaluation_review"}) #리뷰 영역. 별점, 내용


rates = contents_div.find_all(name="span", attrs={"class": "ico_star inner_star"}) # 별점값 목록
# rates = contents_div.find(name="span", attrs={"class": "ico_star star_rate"}) # 별점값 목록
reviews = contents_div.find_all(name="p", attrs={"class": "txt_comment"}) # 리뷰 목록


In [255]:
review_results = []

for rate, review in zip(rates, reviews): # (별점목록, 리뷰목록)
        # series = pd.Series(row, index=df.columns) # score, review 인덱스 설정. 시리즈 객체로 생성.
        rate = ((int(rate.get("style")[6:-2]) / 100) * 5)
        review = review.find(name="span").text
        review_results.append({"name" : store_name,
                              "rate" : rate,
                              "review" : review})

reviews_df = pd.DataFrame(review_results)

In [257]:
pd.DataFrame(review_results)

,name,rate,review
0,이도곰탕 본점,5.0,
1,이도곰탕 본점,2.0,너무 짜요. 처음엔 먹을만했는데. 절반정도 먹고나선 너무 짜서 짜증날 정도임. 맛 ...
2,이도곰탕 본점,5.0,곰탕 정말 빨리나옵니다. 주문하고 5분도 안되서 음식 받은거 같아요. 정말 맛있고 ...


In [262]:
btnMore2 = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[8]/div[3]/a')
btnMore2.click()

In [263]:
soup = bs(driver.page_source,'lxml')
total_strong = soup.find(name="strong", attrs={"class": "total_evaluation"})
tot_num = total_strong.find(name="span", attrs={"class": "color_b"}) # 등록된 리뷰 갯수
contents_div = soup.find(name='div', attrs={"class": "evaluation_review"}) #리뷰 영역. 별점, 내용


rates = contents_div.find_all(name="span", attrs={"class": "ico_star inner_star"}) # 별점값 목록
# rates = contents_div.find(name="span", attrs={"class": "ico_star star_rate"}) # 별점값 목록
reviews = contents_div.find_all(name="p", attrs={"class": "txt_comment"}) # 리뷰 목록

In [264]:
review_results = []

for rate, review in zip(rates, reviews): # (별점목록, 리뷰목록)
        # series = pd.Series(row, index=df.columns) # score, review 인덱스 설정. 시리즈 객체로 생성.
        rate = ((int(rate.get("style")[6:-2]) / 100) * 5)
        review = review.find(name="span").text
        review_results.append({"name" : store_name,
                              "rate" : rate,
                              "review" : review})

reviews_df = pd.DataFrame(review_results)

In [265]:
pd.DataFrame(review_results)


,name,rate,review
0,이도곰탕 본점,5.0,
1,이도곰탕 본점,2.0,너무 짜요. 처음엔 먹을만했는데. 절반정도 먹고나선 너무 짜서 짜증날 정도임. 맛 ...
2,이도곰탕 본점,5.0,곰탕 정말 빨리나옵니다. 주문하고 5분도 안되서 음식 받은거 같아요. 정말 맛있고 ...
3,이도곰탕 본점,5.0,수육이랑 곰탕먹었어요곰탕은 주문하자마자 빨리 나왔어요고기도 많이 들어가있고간이 되어...
4,이도곰탕 본점,5.0,세종대왕의 이름 (이도 李裪)에 걸맞는 진한 맛의 곰탕입니다마침 제옆에 외국인도 와...
5,이도곰탕 본점,4.0,간단하게 먹기 좋은 듯...
6,이도곰탕 본점,5.0,존맛탱 짱맛있어요!또 올거에요!면도 쌀국수면같아서 맛있어요!주차는 발렛30분3000원
7,이도곰탕 본점,1.0,싸가지없고 양심없는 주차 발렛 때문에 기분 개잡치고 갑니다 갈거면 걸어서가세요


In [268]:
more_nums = (int(tot_num.text) - 3) // 5 + 1
for more_num in tqdm(range(more_nums)):
    btnMore2 = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[8]/div[3]/a')
    btnMore2.click()
    time.sleep(2)

100%|██████████| 34/34 [01:11<00:00,  2.10s/it]


In [269]:
soup = bs(driver.page_source,'lxml')
total_strong = soup.find(name="strong", attrs={"class": "total_evaluation"})
tot_num = total_strong.find(name="span", attrs={"class": "color_b"}) # 등록된 리뷰 갯수
contents_div = soup.find(name='div', attrs={"class": "evaluation_review"}) #리뷰 영역. 별점, 내용


rates = contents_div.find_all(name="span", attrs={"class": "ico_star inner_star"}) # 별점값 목록
# rates = contents_div.find(name="span", attrs={"class": "ico_star star_rate"}) # 별점값 목록
reviews = contents_div.find_all(name="p", attrs={"class": "txt_comment"}) # 리뷰 목록

In [270]:
review_results = []

for rate, review in zip(rates, reviews): # (별점목록, 리뷰목록)
        # series = pd.Series(row, index=df.columns) # score, review 인덱스 설정. 시리즈 객체로 생성.
        rate = ((int(rate.get("style")[6:-2]) / 100) * 5)
        review = review.find(name="span").text
        review_results.append({"name" : store_name,
                              "rate" : rate,
                              "review" : review})

reviews_df = pd.DataFrame(review_results)

In [271]:
reviews_df

,name,rate,review
0,이도곰탕 본점,5.0,
1,이도곰탕 본점,2.0,너무 짜요. 처음엔 먹을만했는데. 절반정도 먹고나선 너무 짜서 짜증날 정도임. 맛 ...
2,이도곰탕 본점,5.0,곰탕 정말 빨리나옵니다. 주문하고 5분도 안되서 음식 받은거 같아요. 정말 맛있고 ...
3,이도곰탕 본점,5.0,수육이랑 곰탕먹었어요곰탕은 주문하자마자 빨리 나왔어요고기도 많이 들어가있고간이 되어...
4,이도곰탕 본점,5.0,세종대왕의 이름 (이도 李裪)에 걸맞는 진한 맛의 곰탕입니다마침 제옆에 외국인도 와...
...,...,...,...
164,이도곰탕 본점,4.0,곰탕
165,이도곰탕 본점,4.0,소수육46. 면+죽 추가10.
166,이도곰탕 본점,2.0,공장형.밥대신 국수.잘 모르겠다.
167,이도곰탕 본점,4.0,


## 정리해서 만들어봄

In [272]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pickle

from datetime import datetime
from datetime import timedelta

pd.set_option('display.max_columns', None) # 데이터프레임 모든 열 출력

import warnings
warnings.filterwarnings(action='ignore')

In [323]:
# pickle load
with open('data/result.pkl', 'rb') as f:
    data = pickle.load(f)

# pickle 파일에서 모범음식점 지정, 신청, 취소 데이터프레임 각각 선언
assigned_df = data["data"]["지정"]
applied_df = data["data"]["신청"]
canceled_df = data["data"]["취소"]

In [324]:
assigned_df.head(3)

,CGG_CODE,ASGN_YY,ASGN_SNO,APPL_YMD,ASGN_YMD,UPSO_NM,SITE_ADDR_RD,SITE_ADDR,PERM_NT_NO,SNT_UPTAE_NM,MAIN_EDF,TRDP_AREA,ADMDNG_NM,GRADE_FACIL_GBN,UPSO_SITE_TELNO
0,3220000,2014,0160,20141006,20141006,이도곰탕,"서울특별시 강남구 논현로94길 29-5, 지상1층,지상2층 (역삼동)",서울특별시 강남구 역삼동 671번지 17호,3220000-101-2001-24197,한식,곰탕,194.41,역삼1동,상수도전용,02 5010738
1,3220000,2021,139,20181001,20211116,솥내음 스타필드 코엑스몰점,"서울특별시 강남구 영동대로 513, 코엑스 지하1층 O-107호 (삼성동)",서울특별시 강남구 삼성동 159번지 코엑스,3220000-101-2020-00643,한식,직화불백,74.56,삼성1동,,
2,3220000,2010,0024,20100210,20100210,스시히로바,"서울특별시 강남구 삼성로 620, 블래스톤리조트 지상1층 (삼성동)",서울특별시 강남구 삼성동 70번지 블래스톤리조트,3220000-101-2002-00383,일식,초밥,216.56,삼성1동,상수도전용,02 5155511


In [329]:
assigned_df["UPSO_NM_MOD"] = assigned_df["UPSO_NM"].apply(lambda x: x.split("코엑스몰")[0])

In [330]:
assigned_df["UPSO_NM_MOD"] = assigned_df["UPSO_NM_MOD"].apply(lambda x: x + " 강남구")

In [331]:
assigned_df["UPSO_NM_MOD"] = assigned_df["UPSO_NM_MOD"].apply(lambda x: x.split("주)")[-1])

In [332]:
assigned_df["UPSO_NM_MOD"]

0             이도곰탕 강남구
1        솥내음 스타필드  강남구
2            스시히로바 강남구
3         서백자 간장게장 강남구
4              봉산집 강남구
            ...       
311           모스가든 강남구
312    10꼬르소꼬모서울카페 강남구
313          현대낙지집 강남구
314             초심 강남구
315    아야진생태찌개 역삼점 강남구
Name: UPSO_NM_MOD, Length: 316, dtype: object

In [333]:
assigned_df["SITE_ADDR_RD_SPLIT"] = assigned_df["SITE_ADDR_RD"].apply(lambda x: x.split(",")[0])

In [334]:
assigned_df["SITE_ADDR_RD_SPLIT"] = assigned_df["SITE_ADDR_RD_SPLIT"].apply(lambda x: x.replace("서울특별시", "서울"))


In [335]:
assigned_df["SITE_ADDR_RD_SPLIT"]

0      서울 강남구 논현로94길 29-5
1         서울 강남구 영동대로 513
2          서울 강남구 삼성로 620
3          서울 강남구 삼성로 542
4          서울 강남구 삼성로 564
              ...        
311     서울 강남구 논현로139길 12
312       서울 강남구 압구정로 416
313     서울 강남구 압구정로14길 11
314     서울 강남구 언주로172길 55
315        서울 강남구 논현로 329
Name: SITE_ADDR_RD_SPLIT, Length: 316, dtype: object

In [336]:
assigned_df.head()

,CGG_CODE,ASGN_YY,ASGN_SNO,APPL_YMD,ASGN_YMD,UPSO_NM,SITE_ADDR_RD,SITE_ADDR,PERM_NT_NO,SNT_UPTAE_NM,MAIN_EDF,TRDP_AREA,ADMDNG_NM,GRADE_FACIL_GBN,UPSO_SITE_TELNO,UPSO_NM_MOD,SITE_ADDR_RD_SPLIT
0,3220000,2014,0160,20141006,20141006,이도곰탕,"서울특별시 강남구 논현로94길 29-5, 지상1층,지상2층 (역삼동)",서울특별시 강남구 역삼동 671번지 17호,3220000-101-2001-24197,한식,곰탕,194.41,역삼1동,상수도전용,02 5010738,이도곰탕 강남구,서울 강남구 논현로94길 29-5
1,3220000,2021,139,20181001,20211116,솥내음 스타필드 코엑스몰점,"서울특별시 강남구 영동대로 513, 코엑스 지하1층 O-107호 (삼성동)",서울특별시 강남구 삼성동 159번지 코엑스,3220000-101-2020-00643,한식,직화불백,74.56,삼성1동,,,솥내음 스타필드 강남구,서울 강남구 영동대로 513
2,3220000,2010,0024,20100210,20100210,스시히로바,"서울특별시 강남구 삼성로 620, 블래스톤리조트 지상1층 (삼성동)",서울특별시 강남구 삼성동 70번지 블래스톤리조트,3220000-101-2002-00383,일식,초밥,216.56,삼성1동,상수도전용,02 5155511,스시히로바 강남구,서울 강남구 삼성로 620
3,3220000,2016,0089,20161125,20161125,서백자 간장게장,"서울특별시 강남구 삼성로 542, 지상2층 (삼성동, 석천빌딩)",서울특별시 강남구 삼성동 151번지 4호 석천빌딩 2층,3220000-101-2012-00416,중국식,간장게장,260.30,삼성1동,,02 34536008,서백자 간장게장 강남구,서울 강남구 삼성로 542
4,3220000,2016,0084,20161125,20161125,주)봉산집,"서울특별시 강남구 삼성로 564, 지상2층,지상3층 (삼성동)",서울특별시 강남구 삼성동 145번지 19호,3220000-101-2011-01048,한식,차돌박이,452.95,삼성1동,,,봉산집 강남구,서울 강남구 삼성로 564


In [287]:
# 브라우저 할당 
driver = wb.Chrome()
driver.get('https://map.kakao.com/')
# driver.maximize_window()

# 파란색 팝업 창 기억나는가?
    # 이 부분을 사용자가 클릭해서 없애듯, 
    # 컴퓨터에게도 똑같이 시켜줘야 한다.
    
click_blue_popup = driver.find_element(By.CSS_SELECTOR,'body > div.coach_layer.coach_layer_type1 > div > div > div > span')
click_blue_popup.click()

# 검색창 지정 (마우스로 올려놓기)

search = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.query')

# 검색어 입력('이수역 맛집')

store_name = '이도곰탕'

search.send_keys(store_name)

# ENTER키 누르고 데이터가 로드할 수 있는 시간까지 할당해주기

search.send_keys(Keys.ENTER)
time.sleep(2)

# BeautifulSoup으로 객체화
# 보다 더 빠른 데이터 수집을 위해 -> 추출하는 시간도 이제는 고려

soup = bs(driver.page_source,'lxml')

# 상세보기
moreviews = soup.find_all(name="a", attrs={"class":"moreview"})

# 가장 먼저 나오는 검색 결과의 page url 저장
page_url = moreviews[0].get("href")
# driver.close()
# print(page_url)

In [290]:
addresses = soup.select('div.addr > p:nth-child(1)')

In [322]:
for i, address in enumerate(addresses):
    # print(i, address.text)
    print(i, address.text)

    if "서울 강남구 논현로94길 29-5" in address.text:
        ii = i
    else:
        print("없음")

print(ii)

0 서울 중구 다동길 30
없음
1 서울 종로구 계동길 33-8
없음
2 서울 강남구 논현로94길 29-5
3 서울 서초구 신반포로 176 1층 111-1호
없음
4 서울 송파구 법원로 127 문정대명벨리온 105~107호
없음
5 경기 고양시 일산서구 킨텍스로 171 2층
없음
6 서울 강남구 역삼동 671-14
없음
2
